In [2]:
%pip install intel-extension-for-pytorch==2.2
!pip install transformers==4.35.2
%pip install torch==2.2.0

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /srv/jupyter/python-venv/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /srv/jupyter/python-venv/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import sys
!{sys.executable} -m pip install intel-extension-for-pytorch==2.2 --no-warn-script-location > /dev/null
!{sys.executable} -m pip install transformers==4.35.2 --no-warn-script-location > /dev/null
!{sys.executable} -m pip install torch==2.2.0 --no-warn-script-location > /dev/null



[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /srv/jupyter/python-venv/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /srv/jupyter/python-venv/bin/python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: /srv/jupyter/python-venv/bin/python3 -m pip install --upgrade pip


In [4]:
exit()

In [5]:
import torch
import intel_extension_for_pytorch as ipex
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

In [9]:
Model='Intel/neural-chat-7b-v3-3'
model = AutoModelForCausalLM.from_pretrained(Model)
tokenizer = AutoTokenizer.from_pretrained(Model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [17]:
qconfig = ipex.quantization.get_weight_only_quant_qconfig_mapping(
    weight_dtype = torch.quint4x2, #torch.qint8
    lowp_mode = ipex.quantization.WoqLowpMode.NONE, # FP16, BF16, INT8
)
checkpoint=None
model_ipex=ipex.llm.optimize(model, quantization_config=qconfig, low_precision_checkpoint=checkpoint)
del model

/home/ub0d2875c9f8623d9937577605680443/.local/lib/python3.11/site-packages/intel_extension_for_pytorch/quantization/_quantize.py:97: UserWarning: BatchNorm folding failed during the prepare process.
  warnings.warn("BatchNorm folding failed during the prepare process.")


ipex.llm.optimize is doing the weight only quantization


/home/ub0d2875c9f8623d9937577605680443/.local/lib/python3.11/site-packages/intel_extension_for_pytorch/transformers/models/reference/modules/attentions.py:2017: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if (input_shape[-1] > 1 or self.sliding_window is not None) and self.is_causal:
/home/ub0d2875c9f8623d9937577605680443/.local/lib/python3.11/site-packages/transformers/modeling_attn_mask_utils.py:137: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if past_key_values_length > 0:
/home/ub0d2875c9f8623d9937577605680443/.local/lib/python3.11/site-package

ipex.llm.optimize has set the optimized or quantization model for model.generate()


In [20]:
system_message= """\n\n You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. If you don't know the answer to a question, please don't share false information."""
prompt= "\n\n You are an expert in Kollywood. Can you tell me 5 fun facts about the Tamil Film Industry?"
model_answer_1 = 'None'

prompt_tempate = f"""
### System:
{system_message}

### User:
{prompt}

### Assistant:
"""

inputs = tokenizer(prompt_tempate, return_tensors="pt").input_ids

In [21]:
streamer = TextStreamer(tokenizer,skip_prompt=True)

with torch.inference_mode():
    tokens = model_ipex.generate(
        inputs,
        streamer=streamer,
        max_new_tokens=300,
        repetition_penalty=1.5,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Here are some interesting tidbits from the world of South Indian cinema (Telugu/Kannada - "Sandal" films; Malayali- "Mallyuplam"; Marathi – "Maratheychi Goshta", but not so much for now) or more specifically on 'Chennaivarmanchis', i.e., The TAMIL FIlm industry popularly known as ‘’kOLLYWOOD” :
1.) Ajith Kumasan is one among few actors who has never won any major awards at SIIMAs(South India International Movie Awards), yet he holds multiple Guinness World Records including “Highest number of tickets sold within three hours for first day, First show”. He also owns record for most movie releases by an actor with his film Billa Suruli getting released simultaneously across 204 screens worldwide without prior publicity. His fans call him Thalaiva which means leader implying that they consider themselves under their own leadership when watching movies featuring this star. This shows how strong fan following can be even if official recognition isn't there always.

2.) Kamal Haasan was once